In [1]:
!pip install --upgrade pip

In [2]:
%%bash
ipython kernel install --user --name=.venv_hls4ml

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.


Installed kernelspec .venv_hls4ml in /home/usrp-gnu/.local/share/jupyter/kernels/.venv_hls4ml


In [3]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu


In [4]:
!pip install hls4ml[profiling]

In [5]:
pip install scikit-learn pydot graphviz

Note: you may need to restart the kernel to use updated packages.


In [6]:
import keras
import tensorflow as tf 
import zipfile
import os

2024-09-24 17:57:03.417970: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 17:57:07.670638: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-24 17:57:09.602928: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 17:57:10.535368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 17:57:10.682840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 17:57:12.007365: I tensorflow/core/platform/cpu_feature_gu

In [7]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [8]:
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
#from utils.callbacks import all_callbacks

In [10]:
if os.path.exists('data'):
    print('Folder\'data\' already created. ')
else:
    print('Folder\'data\' created.')
    os.mkdir('data')

Folder'data' already created. 


In [11]:
print(tf.__version__)

2.17.0


In [12]:
pip install tensorflow-model-optimization

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install --upgrade tensorflow tensorflow-model-optimization

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install --upgrade qkeras

Note: you may need to restart the kernel to use updated packages.


In [15]:
print(tf.__version__)

2.17.0


In [16]:
import hls4ml
import yaml

In [17]:
print(hls4ml.__version__)

0.8.1


In [18]:
# Unzip the file
with zipfile.ZipFile('DATASET.zip', 'r') as zip_ref:
    zip_ref.extractall('DATASET')  # Extract the contents to dataset

In [19]:
# Verify that the dataset was extracted
!ls DATASET

Cancer_detection  Normal


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
# Path to the extracted dataset
dataset_path = 'DATASET'

In [22]:
# Create an ImageDataGenerator object
datagen = ImageDataGenerator(rescale=1./255)  # Normalize the images

In [23]:
# Load the dataset from the directory
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(200, 200),  # Resize images to 200x200 pixels
    color_mode='grayscale',  # Convert images to grayscale
    batch_size=32,           # Batch size of 32 images
    class_mode='categorical'  # Assuming you have multiple classes
)

Found 172 images belonging to 2 classes.


In [24]:
print(f"Classes: {train_generator.class_indices}")  # Shows class mapping

Classes: {'Cancer_detection': 0, 'Normal': 1}


In [25]:
print(f"Number of images: {train_generator.samples}")  # Shows the total number of images

Number of images: 172


In [26]:
# Data augmentation to artificially increase dataset size
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Random rotation
    width_shift_range=0.2,  # Horizontal shift
    height_shift_range=0.2, # Vertical shift
    shear_range=0.2,        # Shear transformation
    zoom_range=0.2,         # Zoom in/out
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest'     # Filling missing pixels after transformation
)

train_generator = datagen.flow_from_directory('DATASET',
    target_size=(200, 200),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 172 images belonging to 2 classes.


In [27]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np

In [28]:
# Step 1: Set the path to your dataset (replace with the actual path to your 'train' folder)
dataset_path = 'DATASET'

In [29]:
# Step 2: Create an ImageDataGenerator object
datagen = ImageDataGenerator(rescale=1./255)  # Normalize images

In [30]:
# Step 3: Load the dataset from the folder
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(200, 200),  # Resize images to 200x200 pixels
    color_mode='grayscale',  # Convert images to grayscale
    batch_size=32,           # Load 32 images at a time
    class_mode='categorical'  # Assuming your labels are categorical (for multi-class classification)
)

Found 172 images belonging to 2 classes.


In [31]:
# Step 4: Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 1)),
    MaxPool2D(2),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPool2D(2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')  # Output layer
])

/home/usrp-gnu/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-24 17:58:04.870378: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69337088 exceeds 10% of free system memory.
2024-09-24 17:58:04.905686: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69337088 exceeds 10% of free system memory.
2024-09-24 17:58:04.926365: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69337088 exceeds 10% of free system memory.


In [32]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [46]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 97, 97, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 46, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 33856)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    17,334,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,727,750 (132.48 MB)

 Trainable params: 17,363,874 (66.24 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 17,363,876 (66.24 MB)

In [33]:
# Train the model
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10  # Adjust the number of epochs as needed
)

Epoch 1/10


2024-09-24 17:58:06.667710: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69337088 exceeds 10% of free system memory.
2024-09-24 17:58:07.184900: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69337088 exceeds 10% of free system memory.
/home/usrp-gnu/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.5593 - loss: 2.0053
Epoch 2/10
1/5 ━━━━━━━━━━━━━━━━━━━━ 3s 965ms/step - accuracy: 0.8125 - loss: 0.4807

2024-09-24 17:58:18.252340: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.8125 - loss: 0.4807
Epoch 3/10


...

5/5 ━━━━━━━━━━━━━━━━━━━━ 29s 960ms/step - accuracy: 0.7055 - loss: 0.3444
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.9375 - loss: 0.1252
Epoch 5/10


2024-09-24 17:58:48.543652: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 889ms/step - accuracy: 1.0000 - loss: 0.0439
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0114  
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 1.0000 - loss: 0.0086
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0038  
Epoch 9/10


2024-09-24 17:59:02.291644: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 786ms/step - accuracy: 1.0000 - loss: 0.0046
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 1.0000 - loss: 0.0015  


In [34]:
# Step 6: Train the model with the dataset
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10  # Adjust based on your needs
)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 951ms/step - accuracy: 1.0000 - loss: 0.0017
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 1.5173e-04  
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 907ms/step - accuracy: 1.0000 - loss: 9.5698e-04
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 7.2244e-04  
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 1.0000 - loss: 6.1508e-04
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 4.9617e-04  
Epoch 7/10


2024-09-24 17:59:31.543831: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 798ms/step - accuracy: 1.0000 - loss: 3.3651e-04
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 5.6649e-04  
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 844ms/step - accuracy: 1.0000 - loss: 3.0774e-04
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 2.5760e-04 


In [35]:
!pip install opencv-python

In [36]:
import cv2

In [37]:
def remove_white_border(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply binary thresholding
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Get bounding box coordinates
    x, y, w, h = cv2.boundingRect(contours[0])
    # Crop the image to remove the white border
    return image[y:y+h, x:x+w]

In [38]:
# Apply it to each image in the training set
def preprocess_images(image_path):
    img = cv2.imread(image_path)
    img_no_border = remove_white_border(img)
    return img_no_border

In [39]:
# Set the number of images to predict
num_images_to_predict = 100
predicted_labels = []
actual_labels = []
filenames = []

In [40]:
# Initialize counters
current_count = 0

In [41]:
# Loop through batches until you reach the desired number of images
for batch_images, batch_labels in train_generator:
    # Loop through each image in the current batch
    for i in range(len(batch_images)):
        if current_count >= num_images_to_predict:
            break  # Stop if you've reached the desired number of images
        
        # Get the current image and expand dimensions for prediction
        image = np.expand_dims(batch_images[i], axis=0)
        
        # Predict the class (Cancerous or Non-Cancerous)
        prediction = model.predict(image)
        
        # Assuming binary classification (0 = Cancerous, 1 = Non-Cancerous)
        if prediction[0][0] < 0.5:
            predicted_labels.append("Non-Cancerous")
        else:
            predicted_labels.append("Cancerous")
        
        # Store the actual label and filename for reference
        actual_labels.append("Cancerous" if np.argmax(batch_labels[i]) == 0 else "Non-Cancerous")
        filenames.append(train_generator.filenames[train_generator.batch_index * train_generator.batch_size + i])
        
        current_count += 1

    if current_count >= num_images_to_predict:
        break  # Stop once you've predicted 100 images

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

In [42]:
# Display the predictions
for idx in range(num_images_to_predict):
    print(f"Image: {filenames[idx]}, Predicted: {predicted_labels[idx]}, Actual: {actual_labels[idx]}")

Image: Cancer_detection/Tumor- (1034).jpg, Predicted: Non-Cancerous, Actual: Non-Cancerous
Image: Cancer_detection/Tumor- (1035).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (1036).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (1038).jpg, Predicted: Non-Cancerous, Actual: Non-Cancerous
Image: Cancer_detection/Tumor- (1039).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (104).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (1040).jpg, Predicted: Non-Cancerous, Actual: Non-Cancerous
Image: Cancer_detection/Tumor- (1041).jpg, Predicted: Non-Cancerous, Actual: Non-Cancerous
Image: Cancer_detection/Tumor- (1043).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (1044).jpg, Predicted: Non-Cancerous, Actual: Non-Cancerous
Image: Cancer_detection/Tumor- (1045).jpg, Predicted: Cancerous, Actual: Cancerous
Image: Cancer_detection/Tumor- (1046).jpg, Predi

In [43]:
#history = model.fit(train_generator, steps_per_epoch = train_generator.samples //train_generator.batch_size, epochs=10)    

In [44]:
accuracy_values=history.history['accuracy']

mean_accuracy=np.mean(accuracy_values)

print(f"mean accuracy over all epochs: {mean_accuracy:.8f}")

mean accuracy over all epochs: 0.92285714


In [52]:
#y_keras=model.predict(image)
#print("Accuracy: {}".format(accuracy_score(np.argmax(y_keras, axis=1))))
#plt.figure(figsize=(9,9))
#_=plotting.makeRoc(y_test, y_keras, le.classes_)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


TypeError: missing a required argument: 'y_pred'

In [53]:
#with open("keras-config-yml", 'r') as ymlfile:
    #config = yaml.load(ymlfile, Loader=yaml.FullLoader)

FileNotFoundError: [Errno 2] No such file or directory: 'keras-config-yml'